In [1]:
import pandas as pd

df = pd.read_csv('btc_5m.csv')
print(df.index)


RangeIndex(start=0, stop=489208, step=1)


In [36]:
#30분 내로 2% 이상 올랐는지만 check 할까? 
#open_time,open,high,low,close,volume,close_time,qav,not,tbbav,tbqav,ignore
def rise_in_thirty(a,b,c,d,e,target):
  # print(type(a),type(target))

  m = max(a,b,c,d,e)  
  
  answer = m/target-1
  return answer>0.02



In [ ]:
df['answer'] = None
lengt = len(df.index)
for i in range(lengt):
  if(i>= lengt-5):
    break
  #2단위 동안 종가가 하락의 합이 -2% 가 되지 않고, 두 High 값중 2%이상 상승값이 있다.
  df['answer'].iloc[i] = rise_in_thirty(df.iloc[i+1]['high'],df.iloc[i+2]['high'],df.iloc[i+3]['high'],df.iloc[i+4]['high'],df.iloc[i+5]['high'], df.iloc[i]['high'])



50만개의 인덱스를 일일이 찾아다녀야 해서 오래걸렸나보다 20분이나 걸리네 ; 
바로 저장 할까? . 

어차피 자기 자신이 max값이면 false가 되니까 rolling을 사용해서 만드는게 좋겠다.  다음에 쓸일이 있다면 

In [38]:
df.to_csv('btc_target_added.csv')

In [85]:
df = pd.read_csv('btc_target_added.csv')

/var/folders/h9/d46828gd2xs0tn_jvhbwhkz00000gn/T/ipykernel_82742/1129221123.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('btc_target_added.csv')


In [86]:
#이동 평균선을 구하자. 애매하게 6일 60일 로 하자 ; 
df['ma7'] = df['close'].rolling(window=7).mean()
df['ma60'] = df['close'].rolling(window=60).mean()


In [87]:
# 볼린저 밴드. # 밴드의 상단과 현제가 와의 비율만 활용해보자. 기울기를 구해야 하는데 .  기울기 의미 없지?
def get_sma(prices, rate):
  return prices.rolling(rate).mean()

def get_bollinger_band_width(prices, rate=20):
  sma = get_sma(prices, rate) # 이동평균선을 구하지. (DataFrame형식이다)
  std = prices.rolling(rate).std() # 20일간 이동평균선의 표준편차를 구하지.
  bollinger_up = sma + std * 2 # Calculate top band
  # bollinger_down = sma - std * 2 # Calculate bottom band
  # return bollinger_up, bollinger_down
  return bollinger_up/prices -1 

In [105]:
bbr=get_bollinger_band_width(df['close'])
df_b= pd.concat([df,bbr],axis=1)

In [106]:
print(len(bbr), len(df), len(df_b))


489208 489208 489208


In [107]:
df_b.iloc[-120:-100]


,Unnamed: 0,open_time,open,high,low,close,volume,close_time,qav,not,tbbav,tbqav,ignore,answer,ma7,ma60,close
489088,489088,1650324900000,40823.98,40826.85,40721.25,40739.99,177.14494,1650325199999,7.223150e+06,2772,71.63360,2.920283e+06,0.0,False,40846.252857,40743.213167,0.005063
489089,489089,1650325200000,40739.98,40782.73,40730.00,40757.79,121.63944,1650325499999,4.957213e+06,2291,60.94671,2.483775e+06,0.0,False,40828.381429,40744.556000,0.004662
489090,489090,1650325500000,40757.79,40828.24,40757.79,40797.76,137.85260,1650325799999,5.622928e+06,2656,66.39907,2.708422e+06,0.0,False,40822.965714,40746.776167,0.003678
489091,489091,1650325800000,40797.76,40826.68,40775.91,40800.00,124.36442,1650326099999,5.073965e+06,2338,60.15377,2.454033e+06,0.0,False,40810.820000,40749.664500,0.003608
489092,489092,1650326100000,40800.00,40814.04,40777.14,40801.13,88.05192,1650326399999,3.592222e+06,1940,28.22632,1.151493e+06,0.0,False,40793.431429,40752.249833,0.003581
489093,489093,1650326400000,40801.13,40915.02,40792.45,40837.76,472.14456,1650326699999,1.928435e+07,4772,295.73520,1.207751e+07,0.0,False,40794.058571,40755.913333,0.002577
489094,489094,1650326700000,40837.76,40929.52,40817.87,40912.27,118.60441,1650326999999,4.848129e+06,3310,59.97558,2.451685e+06,0.0,False,40806.671429,40761.898333,0.001000
489095,489095,1650327000000,40912.27,41099.39,40846.06,41046.52,510.94234,1650327299999,2.094394e+07,7678,297.34404,1.219161e+07,0.0,False,40850.461429,40770.761167,-0.001205
489096,489096,1650327300000,41046.53,41062.79,40886.64,40903.65,208.74665,1650327599999,8.551341e+06,4646,83.72449,3.429278e+06,0.0,False,40871.298571,40776.515333,0.002416
489097,489097,1650327600000,40903.66,40935.25,40877.26,40888.15,99.27177,1650327899999,4.060559e+06,3014,43.22145,1.767730e+06,0.0,False,40884.211429,40780.856000,0.002843


In [108]:
# df_b.drop(columns=['Unnamed: 0'], inplace=True)
df_b.columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'qav', 'not', 'tbbav', 'tbqav', 'ignore', 'answer', 'ma7', 'ma60', 'bbr']

df_b.columns


Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'qav', 'not', 'tbbav', 'tbqav', 'ignore', 'answer', 'ma7', 'ma60',
       'bbr'],
      dtype='object')

R S I 추가 해보자. 

In [109]:
import numpy as np

In [99]:
date_index = df_b.index.astype('str')# 이게 무슨 행동일까? 데이터프레임 전체나, 인덱스나, 컬럼의 변수타입을 바꿀수 있다. astype()
# date_index = df_b.index

def calcRSI(df, period):
  U=np.where(df['close'].diff(1) > 0, df['close'].diff(1), 0) # numpy.Array 생성  # 차이지만 나중에 나누어주기 때문에 값에 따라 스케일이 다르지 않음. 
  D=np.where(df['close'].diff(1) < 0, df['close'].diff(1)*(-1), 0)
  print(U[:10])
  AU = pd.DataFrame(U, index=date_index).rolling(window=period).mean() #df로 만드는데 index는 이것이 insert될 df의 것을 가져오나보다.
  AD = pd.DataFrame(D, index=date_index).rolling(window=period).mean()
  RSI = AU / (AU+AD) *100

  return RSI

In [138]:
df[['close']].isnull().sum()

close    0
dtype: int64

In [111]:
# if 'RSI' not in(df_b.columns):
#   print('RSI is added')
#   df_b = pd.concat([df_b, calcRSI(df= df_b,period= 14),columns=['RSI'])], axis=1) # fb 컬럼의 마지막 위치에 RSI를 추가한다는 의미죠. 
if 'RSI' not in(df_b.columns):
  print('RSI is added')
  df_b.insert(len(df_b.columns), 'RSI', calcRSI(df_b, 14))

RSI is added
[ 0.    0.    9.    0.   21.23  4.03 60.23  0.    0.   33.  ]


In [140]:
# df_b.iloc[120:180]
# df_b.drop(columns=['RSI'], inplace=True)
#381, 382, 188358, 364856, 364857, 364858, 364859
df_b[435:480]

,open_time,open,high,low,close,volume,close_time,qav,not,tbbav,tbqav,ignore,answer,ma7,ma60,bbr,RSI,ma7r,ma60t7r
435,1503222900000,4155.00,4155.00,4155.00,4155.00,0.000000,1503223199999,0.000000,0,0.000000,0.000000,9516.522715,False,4155.000000,4130.954167,0.014084,0.000000,0.000000,-0.005787
436,1503223200000,4155.00,4155.00,4155.00,4155.00,0.000000,1503223499999,0.000000,0,0.000000,0.000000,9521.127001,False,4155.000000,4131.987500,0.010340,0.000000,0.000000,-0.005539
437,1503223500000,4155.00,4155.00,4155.00,4155.00,0.000000,1503223799999,0.000000,0,0.000000,0.000000,9551.010823,False,4155.000000,4132.993833,0.009807,0.000000,0.000000,-0.005296
438,1503223800000,4155.00,4155.00,4155.00,4155.00,0.000000,1503224099999,0.000000,0,0.000000,0.000000,9550.020095,False,4155.000000,4133.456833,0.009578,0.000000,0.000000,-0.005185
439,1503224100000,4155.00,4155.00,4155.00,4155.00,0.000000,1503224399999,0.000000,0,0.000000,0.000000,9549.389833,False,4155.000000,4133.670833,0.009316,0.000000,0.000000,-0.005133
440,1503224400000,4155.00,4155.00,4155.00,4155.00,0.000000,1503224699999,0.000000,0,0.000000,0.000000,9549.027551,False,4155.000000,4134.170500,0.008357,NaN,0.000000,-0.005013
441,1503224700000,4155.00,4155.00,4155.00,4155.00,0.000000,1503224999999,0.000000,0,0.000000,0.000000,9547.115232,False,4155.000000,4134.384500,0.006336,NaN,0.000000,-0.004962
442,1503225000000,4140.00,4140.00,4140.00,4140.00,0.478191,1503225299999,1979.710740,4,0.000000,0.000000,9544.658132,False,4152.857143,4135.128000,0.009022,0.000000,0.003106,-0.004269
443,1503225300000,4124.92,4142.41,4124.92,4142.41,0.313349,1503225599999,1295.621836,4,0.202185,835.402876,9539.586793,False,4151.058571,4135.911667,0.007070,13.842619,0.002088,-0.003649
444,1503225600000,4109.84,4142.41,4109.84,4142.41,0.148961,1503225899999,613.083996,2,0.026961,111.683516,9559.143355,False,4149.260000,4136.695333,0.005045,13.842619,0.001654,-0.003028


In [114]:
#이제 뭐해야 되? 모르겠어, 이제 러닝해야지 . 더 다듬어야 하네 , abcde의 조합으로 바꿔야 한다; 근데 그정도로 
df_b['ma7r'] = df_b['ma7']/df_b['close'] -1
df_b['ma60t7r'] = df_b['ma60']/df_b['ma7'] -1

In [116]:
df_b[61:80]

,open_time,open,high,low,close,volume,close_time,qav,not,tbbav,tbqav,ignore,answer,ma7,ma60,bbr,RSI,ma7r,ma60t7r
61,1503110700000,4170.19,4184.00,4170.19,4184.00,0.075794,1503110999999,316.376356,2,0.075794,316.376356,9449.002832,False,4171.172857,4122.545500,0.001383,51.433472,-0.003066,-0.011658
62,1503111000000,4184.00,4184.00,4184.00,4184.00,0.204203,1503111299999,854.385352,3,0.204203,854.385352,9455.502338,False,4173.448571,4126.017333,0.001913,54.797024,-0.002522,-0.011365
63,1503111300000,4156.39,4156.39,4156.39,4156.39,0.100000,1503111599999,415.639000,1,0.000000,0.000000,9453.657714,False,4171.780000,4129.029000,0.008575,36.301300,0.003703,-0.010248
64,1503111600000,4156.39,4156.39,4156.39,4156.39,0.000000,1503111899999,0.000000,0,0.000000,0.000000,9452.371753,False,4170.111429,4131.686833,0.008581,43.171860,0.003301,-0.009214
65,1503111900000,4180.00,4180.00,4156.39,4156.39,0.031934,1503112199999,132.926391,5,0.011934,49.798591,9463.176086,False,4168.167143,4134.277500,0.008587,56.230184,0.002834,-0.008131
66,1503112200000,4142.78,4155.56,4131.19,4155.56,0.391815,1503112499999,1622.429043,6,0.270815,1121.152663,9462.915333,False,4166.104286,4135.850500,0.008229,54.821287,0.002537,-0.007262
67,1503112500000,4149.17,4155.56,4120.00,4140.00,4.943839,1503112799999,20415.552305,24,4.463096,18434.882033,9462.419057,False,4161.818571,4137.167167,0.011868,41.683234,0.005270,-0.005923
68,1503112800000,4140.00,4157.65,4138.00,4157.65,2.510042,1503113099999,10402.805026,26,2.116071,8772.553028,9458.117007,False,4158.054286,4138.778000,0.007199,52.738990,0.000097,-0.004636
69,1503113100000,4156.00,4157.65,4156.00,4156.00,0.712218,1503113399999,2960.094133,16,0.471103,1958.020193,9456.448307,False,4154.054286,4139.811333,0.006848,42.382936,-0.000468,-0.003429
70,1503113400000,4121.00,4130.34,4103.52,4116.41,1.068003,1503113699999,4390.955980,7,0.073365,302.714263,9453.060819,False,4148.342857,4140.062333,0.017709,28.261235,0.007757,-0.001996


In [117]:
mydf = df_b[['ma7r', 'ma60t7r', 'bbr', 'RSI', 'answer']].copy()

In [ ]:
mydf[60:80]


In [121]:
#일단 백업;;
mydf.to_csv('btc_v1.csv')

In [122]:
#이제 하나의 컬럼으로 합쳐야되.
#어떡하지. 단순히 구간을 5개로 나눌 수 있고 
#순서대로 끈어서 구간을 매길 수도 있지.  

mydf.describe()

,ma7r,ma60t7r,bbr,RSI
count,489202.000000,489149.000000,489189.000000,489188.000000
mean,-0.000003,-0.000043,0.006946,50.489953
std,0.003785,0.010126,0.010429,14.850882
min,-0.183621,-0.141461,-0.038126,0.000000
25%,-0.001258,-0.003575,0.001833,40.248644
50%,-0.000039,-0.000250,0.004004,50.500558
75%,0.001179,0.003098,0.008160,60.756978
max,0.144950,0.234609,0.394526,100.000000


In [123]:
na1 = np.array(mydf['ma7r'])

In [126]:
# 총길이는 489208인데 na를 제거할꺼거덩. 
mydf.isnull().sum()

ma7r        6
ma60t7r    59
bbr        19
RSI        20
answer      5
dtype: int64

In [127]:
mydf = mydf.dropna(subset=['ma60t7r', 'answer'])


In [128]:
mydf.isnull().sum()

ma7r       0
ma60t7r    0
bbr        0
RSI        7
answer     0
dtype: int64

In [135]:
mydf.reset_index(inplace=True)
mydf.loc[pd.isna(mydf["RSI"]), :].index

Int64Index([381, 382, 188358, 364856, 364857, 364858, 364859], dtype='int64')

In [141]:
mydf.dropna(subset=['RSI'], inplace=True)

In [ ]:
mydf.isnull().sum()

In [143]:
mydf.head()

,index,ma7r,ma60t7r,bbr,RSI,answer
0,59,-0.001394,-0.011646,0.004659,56.615478,False
1,60,-0.000796,-0.011461,0.004391,39.609287,False
2,61,-0.003066,-0.011658,0.001383,51.433472,False
3,62,-0.002522,-0.011365,0.001913,54.797024,False
4,63,0.003703,-0.010248,0.008575,36.301300,False


In [184]:
x  = np.array(mydf['RSI'])

In [185]:
len(x)
[489137*0.2, 489137*0.4, 489137*0.6, 489137 * 0.8,]

[97827.40000000001, 195654.80000000002, 293482.2, 391309.60000000003]

In [186]:
x =np.sort(x)

In [187]:
#이거 그냥 순서대로 쭉 늘려놓고 5구간으로 나누자. 
print(x[97827],x[195654],x[293482],x[391309])

37.709137709135945 46.6455094594184 54.369713415002686 63.23964382500987


In [196]:
def combine_into_one(ma7, ma60, bb, rsi):
  ans = ''
  if ma7 < -0.00165:
    ans = 'a'
  elif ma7 < -0.000457:
    ans = 'b'
  elif ma7 < -0.000379:
    ans = 'c'
  elif ma7 < 0.001567:
    ans = 'd'
  else:
    ans = 'e'
  #
  if ma60 < 0.001453:
    ans = ans+'a'
  elif ma60 < 0.003033:
    ans = ans+'b'
  elif ma60 < 0.005251:
    ans = ans+'c'
  elif ma60 < 0.009748:
    ans = ans+'d'
  else:
    ans = ans+'e'
    #
  if bb < 0.00145:
    ans = ans+'a'
  elif bb < 0.00303:
    ans = ans+'b'
  elif bb < 0.00525:
    ans = ans+'c'
  elif bb < 0.00974:
    ans = ans+'d'
  else:
    ans = ans+'e'
    #37.709137 46.645509 54.3697134150 63.239643825
  if rsi < 37.7091:
    ans = ans+'a'
  elif rsi < 46.64:
    ans = ans+'b'
  elif rsi < 56.36:
    ans = ans+'c'
  elif rsi < 69.4:
    ans = ans+'d'
  else:
    ans = ans+'e'
  return ans

In [197]:
#ma7r	ma60t7r	bbr	RSI
combination = mydf.apply(lambda x : combine_into_one(ma7=x['ma7r'],ma60=x['ma60t7r'],bb=x['bbr'],rsi=x['RSI']),axis=1)

In [200]:
print(combination.head())
print(len(combination), len(mydf))

0    bacd
1    bacb
2    aaac
3    aabc
4    eada
dtype: object
489137 489137


In [201]:
c2 = pd.concat([combination, mydf['answer']],axis=1)

In [204]:
c2.to_csv('btc_done.csv')